In [ ]:
from HTML import config as Config
from HTML.dataset import *
from HTML.config import ratio, nominal, ordinal, meaningless
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
x_train = pd.read_csv(Config.train_path)
x_train.head()

In [ ]:
x_train.columns

In [ ]:
x_train['is_canceled'].value_counts()

In [ ]:
dummies = pd.get_dummies(
        data=x_train[set(x_train.columns) - {'adr', 'reservation_status', 'reservation_status_date', 'is_canceled'}],
        columns=set(nominal+ordinal) - {'adr', 'reservation_status', 'reservation_status_date', 'is_canceled'},
        drop_first=True,
        dummy_na=True)
dummies = dummies[sorted(set(dummies.columns) - {'index', 'ID'} - set(meaningless))]

In [ ]:
x = dummies[sorted([c for c in dummies.columns if 'reservation_status' not in c])].to_numpy()
# fact = pd.factorize(x_train['is_canceled'])
y = x_train['is_canceled'].to_numpy()

In [ ]:
x.shape

In [ ]:
y.shape

In [ ]:
from sklearn.model_selection import cross_val_score
from xgboost import XGBClassifier

In [ ]:
def CreateBalancedSampleWeights(y_train, largest_class_weight_coef):
    classes = np.unique(y_train)
    classes.sort()
    class_samples = np.bincount(y_train)
    total_samples = class_samples.sum()
    n_classes = len(class_samples)
    weights = total_samples / (n_classes * class_samples * 1.0)
    class_weight_dict = {key : value for (key, value) in zip(classes, weights)}
    class_weight_dict[classes[1]] = class_weight_dict[classes[1]] * largest_class_weight_coef
    sample_weights = [class_weight_dict[y] for y in y_train]

    return sample_weights

In [ ]:
largest_class_weight_coef=np.max(np.bincount(y)) / len(y)
weights = CreateBalancedSampleWeights(y, largest_class_weight_coef=largest_class_weight_coef)

In [ ]:
s = []
for i in [1,3,4,5,6]:
    clf = XGBClassifier(
        objective='binary:logistic',
        n_estimators=1,
        learning_rate=.3,
        max_depth=i,
        subsample=.3,
        n_jobs=6,
        colsample_bytree=.9,
        random_state=1126,
        gamma=10,
    )
    scores = cross_val_score(clf, x, y, cv=5, scoring='f1', verbose=1, n_jobs=2)
    s.append(scores)
    print(np.mean(scores))

In [ ]:
plt.plot([1,5,10,50,100,500], np.mean(np.array(s),axis=1))

In [ ]:
np.mean(np.array(s),axis=1)

In [ ]:
clf = XGBClassifier(
    objective='binary:logistic',
    n_estimators=10,
    learning_rate=.3,
    max_depth=6,
    subsample=.3,
    n_jobs=6,
    colsample_bytree=.9,
    random_state=1126,
    gamma=10
)

In [ ]:
clf

In [ ]:
clf.fit(x, y)

In [ ]:
pred = clf.predict(x)

In [ ]:
mat = np.zeros((3,3))
for p, a in zip(pred, y):
    mat[p, a] += 1

In [ ]:
p

In [ ]:
mat